In [1]:
# settings.py
from dotenv import load_dotenv
load_dotenv()

# OR, the same with increased verbosity
load_dotenv(verbose=True)

# OR, explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

True

In [2]:
import tweepy
import webbrowser
import time
import pandas as pd

In [3]:
import os
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host = os.getenv('aws_host'),
                                         database = os.getenv('aws_schema'),
                                         user = os.getenv('aws_user'),
                                         password = os.getenv('aws_pass'))
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Connected to MySQL Server version  8.0.20
You're connected to database:  ('DORIS_DB',)
MySQL connection is closed


In [4]:
consumer_key = os.getenv("consumer_key")
consumer_secret = os.getenv("consumer_secret")
callback_uri = 'oob'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_uri)
api = tweepy.API(auth)

In [5]:
def extract_timeline_as_df(timeline_list):
    tweets_data = []
    header_cols = ["id_str", "author", "text", "in_reply_to_user_id", "in_reply_to_status_id", "quoted_status_id", "retweet_count", "favorite_count", "created_at"]

    for status in timeline_list:
        status_dict = dict(vars(status))
        keys = status_dict.keys()
        single_tweet_data = {}
        for k in keys:
            if k == "author":
                single_tweet_data[k] = status.author.screen_name
            else:
                single_tweet_data[k] = status_dict[k]
            
        tweets_data.append(single_tweet_data)
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

In [6]:
user = api.get_user("realDonaldTrump")
user_timeline = user.timeline()

In [30]:
def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    sortedtweets = []
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print(f"...{len(alltweets)} tweets downloaded so far")
        
    for tweet in alltweets:
        if not hasattr(tweet, 'quoted_status_id'):
            tweet.quoted_status_id = None
        sortedtweets.append([tweet.id, tweet.user.id, tweet.text, tweet.in_reply_to_status_id, tweet.in_reply_to_user_id, tweet.quoted_status_id, tweet.retweet_count, tweet.favorite_count, tweet.created_at])
    return sortedtweets
    
#     #transform the tweepy tweets into a 2D array that will populate the csv 
#     outtweets = [[tweet.id_str, tweet.created_at, tweet.text] for tweet in alltweets]
    
#     #write the csv  
#     with open(f'new_{screen_name}_tweets.csv', 'w') as f:
#         writer = csv.writer(f)
#         writer.writerow(["id","created_at","text"])
#         writer.writerows(outtweets)
    
#     pass


In [54]:
df5 = pd.DataFrame(get_all_tweets("kamalaharris"), columns=['tweetID', 'userID', 'tweetText', 'inReplyStatus', 'inReplyUser', 'quotedStatusID', 'retweetCount', 'favoriteCount', 'createdAt'])

getting tweets before 1302296545442832383
...400 tweets downloaded so far
getting tweets before 1295930870403076096
...600 tweets downloaded so far
getting tweets before 1287927758232997888
...800 tweets downloaded so far
getting tweets before 1278854471133601791
...1000 tweets downloaded so far
getting tweets before 1269799809084542978
...1200 tweets downloaded so far
getting tweets before 1260609505743835137
...1400 tweets downloaded so far
getting tweets before 1250908833221926913
...1599 tweets downloaded so far
getting tweets before 1239675315825377282
...1799 tweets downloaded so far
getting tweets before 1219812527816744961
...1999 tweets downloaded so far
getting tweets before 1197367513136254975
...2199 tweets downloaded so far
getting tweets before 1190447045703692287
...2398 tweets downloaded so far
getting tweets before 1183737639222026245
...2598 tweets downloaded so far
getting tweets before 1176572512945266688
...2798 tweets downloaded so far
getting tweets before 116925

In [55]:
df5.head()

,tweetID,userID,tweetText,inReplyStatus,inReplyUser,quotedStatusID,retweetCount,favoriteCount,createdAt
0,1311051909055369219,30354991,"Because of Justice Ginsburg, people could get ...",NaN,NaN,NaN,1563,7701,2020-09-29 21:15:07
1,1311035104597245953,30354991,RT @JoeBiden: It’s grassroots donors like you ...,NaN,NaN,NaN,1121,0,2020-09-29 20:08:21
2,1311029142851383296,30354991,More than one million Americans have already v...,NaN,NaN,NaN,1641,6810,2020-09-29 19:44:40
3,1310998786915868672,30354991,"Vote as if your life, your choice, depends on ...",NaN,NaN,NaN,8267,38199,2020-09-29 17:44:02
4,1310993136383143936,30354991,RT @truth: Truth — and we can’t stress this en...,NaN,NaN,NaN,8430,0,2020-09-29 17:21:35


In [56]:
def insert_into_table(mySql_insert_query):    
    try:
        connection = mysql.connector.connect(host = os.getenv('aws_host'),
                                             database = os.getenv('aws_schema'),
                                             user = os.getenv('aws_user'),
                                             password = os.getenv('aws_pass'))
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)

            cursor = connection.cursor()
            for i,row in df5.iterrows():
                cursor.execute(mySql_insert_query, tuple(row))
                connection.commit()
            print(cursor.rowcount, "Success")

    except Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if (connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [57]:
mySql_insert_query = """INSERT IGNORE INTO TWEET (tweetID, userID, tweetText, inReplyStatus, inReplyUser, quotedStatusID, retweetCount, favoriteCount, createdAt)
                                    VALUES (%s , %s, %s, %s, %s, %s, %s, %s, %s) """

df5 = df5.where(pd.notnull(df5), None)
insert_into_table(mySql_insert_query)

Connected to MySQL Server version  8.0.20
1 Success
MySQL connection is closed
